In [73]:
import pandas as pd
import time
from datetime import datetime
import warnings
import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [103]:
home_page = 'https://www.espn.com/nba/playbyplay?gameId=401247147'
options = webdriver.ChromeOptions()
options.add_argument("--headless")
# options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options,executable_path='/Users/joepeirson/Downloads/Chromedriver_folder/chromedriver')
actionChains = ActionChains(driver)
driver.get(home_page)
match_time_data = []
team_data = []
play_data = []
score_data = []
quarter_data = []
driver.implicitly_wait(2)
driver.find_element_by_xpath("//button[contains(@id, 'accept-btn')]").click()
driver.implicitly_wait(2)
collapsed_quarter_tabs = len(driver.find_elements_by_xpath("//*[@class='accordion-item']//a[contains(@href, 'quarter') and contains(@class, 'collapsed')]"))+1
for quarters in range(1,collapsed_quarter_tabs):
    time.sleep(0.25)
    driver.find_element_by_xpath("//*[@class='accordion-item']//a[contains(@href, 'quarter') and contains(@class, 'collapsed')]/div").click()
for quarter in range(1,5):
    lines_of_data = len(driver.find_elements_by_xpath('(//*[@class="accordion-item"])['+str(quarter)+']//*[@class="time-stamp"]'))+1
    for data in range(1,lines_of_data):
        match_time = driver.find_element_by_xpath('((//*[@class="accordion-item"])['+str(quarter)+']//*[@class="time-stamp"])['+str(data)+']').text
        team = driver.find_element_by_xpath('((//*[@class="accordion-item"])['+str(quarter)+']//*[@class="logo"]/img)['+str(data)+']').get_attribute('src').rsplit('/',1)[1].split('.',1)[0]
        play = driver.find_element_by_xpath('((//*[@class="accordion-item"])['+str(quarter)+']//*[@class="game-details"])['+str(data)+']').text
        score = driver.find_element_by_xpath('((//*[@class="accordion-item"])['+str(quarter)+']//*[@class="combined-score"])['+str(data)+']').text
        match_time_data.append(match_time)
        team_data.append(team)
        play_data.append(play)
        score_data.append(score)
        quarter_data.append(quarter)
driver.quit()


In [104]:
df = pd.DataFrame([quarter_data,match_time_data,team_data,play_data,score_data]).transpose()
df.columns = ['q','time','team','play','score']
df

,q,time,team,play,score
0,1,12:00,mia,Daniel Theis vs. Bam Adebayo (Jae Crowder gain...,0 - 0
1,1,11:43,mia,Jae Crowder misses 25-foot three point jumper,0 - 0
2,1,11:42,bos,Jaylen Brown defensive rebound,0 - 0
3,1,11:24,bos,Jayson Tatum misses 15-foot pullup jump shot,0 - 0
4,1,11:23,mia,Heat defensive team rebound,0 - 0
...,...,...,...,...,...
442,4,26.2,bos,Celtics Full timeout,113 - 125
443,4,20.9,bos,Kemba Walker misses 27-foot three point pullup...,113 - 125
444,4,18.9,mia,Duncan Robinson defensive rebound,113 - 125
445,4,0.0,mia,End of the 4th Quarter,113 - 125


In [105]:
player = 'Jayson Tatum'
df.loc[((df['play'].str.startswith(player+' misses')) |
       (df['play'].str.contains('blocks '+player)) |
       (df['play'].str.startswith(player+' makes'))) &
       ~(df['play'].str.contains('free throw')) #removes all free throws
      ]
# this finds all the shooting plays (minus free throws) for Jayson Tatum

,q,time,team,play,score
3,1,11:24,bos,Jayson Tatum misses 15-foot pullup jump shot,0 - 0
8,1,11:09,bos,Jae Crowder blocks Jayson Tatum 's 6-foot driv...,0 - 2
23,1,9:47,bos,Jayson Tatum misses 26-foot three point jumper,2 - 6
36,1,8:38,bos,Jayson Tatum misses 26-foot three point jumper,4 - 6
94,1,52.3,bos,Jayson Tatum misses 14-foot step back jumpshot,24 - 33
121,2,9:39,bos,Jayson Tatum misses driving floating jump shot,35 - 40
125,2,9:08,bos,Jayson Tatum misses layup,35 - 40
138,2,8:29,bos,Jayson Tatum makes 27-foot three pointer (Kemb...,38 - 40
161,2,5:53,bos,Jayson Tatum makes 3-foot dunk,44 - 46
163,2,5:20,bos,Jayson Tatum makes 8-foot driving floating jum...,46 - 49
